In [1]:
import pandas as pd
import json
import re
import nltk
from nltk.tokenize import TweetTokenizer
from nltk import FreqDist
import string

In [2]:
user_demo_profiles_path = './data/User demo profiles.json'
labeled_tweets_path = './data/Twitter_User_Handles_labeled_tweets.json'
labeled_users = './data/labeled_users.csv'

df_tweets = pd.read_json(labeled_tweets_path, orient='index')
df_profiles = pd.read_json(user_demo_profiles_path, orient='values')

In [3]:
df_profiles

,id,name,screen_name,description,lang,img_path
0,12488,Chad Boyce,djsnipa1,"Multimedia Developer, Graphic Designer, DJ, an...",NA,profile pics/60147.jpeg
1,719703,Tomato 😷,Tomato,🇭🇰Rise Up!,NA,profile pics/60148.jpeg
2,722153,blank,blank,"Someone who thinks too much, acts too little.",NA,profile pics/60149.jpeg
3,749003,A ring made from a spoon --- E83.31,IYQbd,Has the kind of luck that turns a professional...,NA,profile pics/60150.jpeg
4,1896301728,عبدالله,755531,,NA,profile pics/60151.jpeg
...,...,...,...,...,...,...
3604,4895390642,Dalton's Eyeliner,FlyAwayEyeliner,"I'm Dalton's eyeliner, who are you? Are you Da...",NA,profile pics/64267.jpeg
3605,4895831833,Jamie Loftus,cowboyjamie77,"41 year old male. interests include learning, ...",NA,profile pics/64268.jpeg
3606,4921995243,saved,memelady,,NA,profile pics/64269.jpeg
3607,4924158634,Winter da CoffeeCat,WinterStar21,"Big fan of cats, coffee, horror stuff and anim...",NA,profile pics/64270.jpeg


In [4]:
df_tweets.index.name = 'user_id'

In [5]:
df_tweets

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
user_id,,,,,,,,,,,,,,,,,,,,,
12488,"YKAR, a futuristic sans serif font by @Emmeran...",@MBonvoyAssist Who can I contact about the ver...,@SSlnes I’d like to win!,@LuckyDucksNFT @Rydog,Now I'm heading to B1000th Floor! #quickrogue,Check this out! https://t.co/eOtMJ99jE7,@AskLyft Thank you! Yes I'm doing it now,@AskLyft Please help me! I got scammed and nee...,@catalogscanner #AnimalCrossing #ACNH #Nintend...,@catalogscanner #AnimalCrossing #ACNH #Nintend...,...,@BandaruOrganics Hello! Will you get these in ...,Evernote helps me remember everything. Get it ...,FINAL FANTASY ALL THE BRAVEST is here! An easy...,My MIR1 flying from San Francisco to Phoenix i...,@MikeRogers0 I love LivePage! Thank you so muc...,I just signed up for @CopyApp and got 5 GB fre...,Say hello to the future of e-mail at http://t....,Check out my personalized 2012 Gaming Summary ...,Mark Bale feat. Cosmo Klein - Gimme Some Love ...,Matty Menck - Rain (Original Mix) http://t.co/...
719703,"In other words, it’s good news about the vacci...",People forget that Trump didn’t even want to s...,The Taiwan government’s response to COVID is s...,#ACNH https://t.co/cEUo84rfvV,Quite happy with yesterday's work on my zen ga...,"Yikes. It isn’t easy to comprehend, as with al...",It’s harder and harder to get excited about bu...,Excellent news. I’ve been feeling so exhausted...,What a line. https://t.co/CnWN9lx4uS,"As a HKer, I’ve gotten used to disappointments...",...,This #iPhone6′s vibration is weak. I got a cal...,@stewart Young people in Hong Kong (myself inc...,Now I really have to go to New York in the nex...,"Wow, congrats @StephenAtHome. Very mixed feeli...",Glitch was a wonderful game and I really miss ...,I was in the game until the final few minutes....,"So, bringing my MacBook, I went to the hotel l...",But the flight was majorly delayed! Thank God ...,I was going to miss the last hours of the game...,Tonight I'm getting pretty emotional when I th...
749003,would it be fair to call lil nas x the first s...,@BillyKumo He did,Decided to only engage with Batman and Batman ...,I'm reading a 1953 Mad magazine Batman parody ...,"Happy Batman Day, Batman people!\nBatman!",Because it's not constrained by the suits the ...,In the second half Robbie declares that his ca...,iffy https://t.co/GnfMHs3h06,I forgot the second wink which makes the first...,"Spike hires a campaign manager, Olio Slick (😉)...",...,ha canadian mathen. i get it https://t.co/crx2...,@madmup https://t.co/XvnNzVcw5v,This whole run feels like it wants to be that ...,This page layout makes no sense to me.\n\nI wa...,There are 911 stans https://t.co/BnRXVrsbb6,"I triggered the ""do you really want to tweet t...",@MnDPS_DPS @MnDPS_MSP Get f.ucked,This is dystopic https://t.co/QreOku3Zit,This is neat. The motion lines show the monste...,Love\nAngel\nMovie\nBaby https://t.co/L2efpaqKrh
822540,@Asmongold ❤️ 🙏,@Hunter4J @Wario64 bonk,@FF_XIV_EN #FFXIV400kSweepstakes,@DaysUntilEW Nice,@AndrewMFanco @jdflynn Ed McBoy,@Wario64 OOS. getting real tired of GameStop t...,"@Wario64 XSX is sold out, was removed from car...","@Wario64 I have pro, can't add to cart",@pixelpipes @dark1x Wish they had used that al...,@WBNG12News RIP Squishy ♥️,...,@kazoodac @Wario64 Stacking 20% off with $10 c...,@Patriotssuck19 @NYRangers lol,@nealmchale @RickCarpiniello Tarasenko or Jade...,@RickCarpiniello The tradition of Festivus beg...,"@Wario64 Glad I spotted this, finally opened m...","@RickCarpiniello You too, Carp.",@MoadhtheGreat @Wario64 I was shocked at how n...,@Wario64 I remember when Newegg was known for ...,@djdresden Yub Nub,@BinghamtonNow What happened to staying 6 feet...
865071,@robo_james How about pizza dipped in water 🤦🏻‍♂️,Day 21 of Quarantine: We installed a trapeze i...,"I love Portuguese egg tarts too, but is this r...","Well, that was an unexpected memory from the r...","The #GDPR has many benefits, but needing to co...",Why does nobody talk about the f

In [6]:
df_users = pd.read_csv(labeled_users)

In [7]:
df_users.head()

,user_id,is_female,year_born,race
0,12488.0,0.0,1980.0,4.0
1,719703.0,0.0,1985.0,4.0
2,722153.0,1.0,1973.0,3.0
3,749003.0,0.0,1982.0,5.0
4,755531.0,0.0,1982.0,4.0


### Preprocssing Text
- all lowercase
- remove urls
- remove @user
- remove #hashtag
- remove punctuations 
- split dataset to train and validation

In [8]:
# A function to preprocessing a column of tweets, the input should be df_tweets['i'], i = 0,...,99
def cleaned_tweet(tweets):
    #lower case
    tweets = tweets.str.lower()
    #remove urls
    tweets = tweets.apply(lambda x: re.sub(r'https?:\/\/\S+', '', str(x)))
    # remove url/website that didn't use http, is only checking for .com websites 
    # so words that are seperated by a . are not removed
    tweets = tweets.apply(lambda x: re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", '', x))
    # remove @user
    tweets = tweets.apply(lambda x: re.sub(r'@[A-Za-z0-9_]+', '', x))
    # remove #hashtag
    tweets = tweets.apply(lambda x: re.sub(r'#[A-Za-z0-9_]+', '', x))
    # remove special characters, numbers, punctuations
    tweets = tweets.apply(lambda x: re.sub(r'\W+', ' ', x))
    # remove short words length less than 3
    tweets = tweets.apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))
    # remove Re-Tweet
    tweets = tweets.apply(lambda x: re.sub(r'^RT[\s]+', '', x))
    return tweets

In [9]:
for col in df_tweets.columns:
    df_tweets[col] = cleaned_tweet(df_tweets[col])
df_tweets

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
user_id,,,,,,,,,,,,,,,,,,,,,
12488,ykar futuristic sans serif font,who can contact about the very rude and poor s...,like win,,now heading b1000th floor,check this out,thank you yes doing now,please help got scammed and need help finding ...,,,...,hello will you get these 00g please sono wood ...,evernote helps remember everything get now for...,final fantasy all the bravest here easy play a...,mir1 flying from san francisco phoenix pocket ...,love livepage thank you much for developing re...,just signed for and got free online storage,say hello the future mail follow for hot news ...,check out personalized 2012 gaming summary fro...,mark bale feat cosmo klein gimme some love ori...,matty menck rain original mix
719703,other words good news about the vaccine safety...,people forget that trump didn even want sign t...,the taiwan government response covid simply ou...,,quite happy with yesterday work zen garden jus...,yikes isn easy comprehend with all speeches fr...,harder and harder get excited about buying new...,excellent news been feeling exhausted this out...,what line,hker gotten used disappointments like this fro...,...,this vibration weak got call while pants pocke...,young people hong kong myself included will no...,now really have new york the next months and a...,wow congrats very mixed feelings right now goi...,glitch was wonderful game and really miss stil...,was the game until the final few minutes could...,bringing macbook went the hotel lobby use thei...,but the flight was majorly delayed thank god f...,was going miss the last hours the game because...,tonight getting pretty emotional when think ab...
749003,would fair call lil nas the first successful o...,did,decided only engage with batman and batman adj...,reading 1953 mad magazine batman parody harvey...,happy batman day batman people batman,because not constrained the suits the furnitur...,the second half robbie declares that his campa...,iffy,forgot the second wink which makes the first w...,spike hires campaign manager olio slick his op...,...,canadian mathen get,,this whole run feels like wants that hawkeye r...,this page layout makes sense want read green f...,there are 911 stans,triggered the you really want tweet that proba...,get ucked,this dystopic,this neat the motion lines show the monster fa...,love angel movie baby
822540,,bonk,,nice,mcboy,oos getting real tired gamestop today,xsx sold out was removed from cart after multi...,have pro can add cart,wish they had used that album art for quite op...,rip squishy,...,stacking off with credits for high profile rel...,lol,tarasenko jaden schwartz would have been nice ...,the tradition festivus begins with the airing ...,glad spotted this finally opened sealed copy a...,you too carp,was shocked how not fun the first hours this g...,remember when newegg was known for their great...,yub nub,what happened staying feet apart from other pe...
865071,how about pizza dipped water,day quarantine installed trapeze the living ro...,love portuguese egg tarts too but this really ...,well that was unexpected memory from the raver...,the has many benefits but needing constantly a...,why does nobody talk about the fact that donke...,basically put the info one needs steal identit...,the start,the start village and ready run the fort wadsw...,,...,,cheering the swiss the williamsburg,translation the password you chose too difficu...,alicia busts open our shipping container carls...,who that,wow sabía tampoco que había estado coma por cu...,did leave prison review app running again,just passed sing sing the train heading north ...,omigod omigod omigod,yvr jfk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3323699762,,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3352812676,women guide burn fat and build muscle the holy...,wondering what now that the weather turning co...,th

In [10]:
# Combine the race and age label to the preprocessed tweets
df_cleanedtweets = pd.merge(df_tweets, df_users, on="user_id")
df_cleanedtweets

,user_id,0,1,2,3,4,5,6,7,8,...,93,94,95,96,97,98,99,is_female,year_born,race
0,12488,ykar futuristic sans serif font,who can contact about the very rude and poor s...,like win,,now heading b1000th floor,check this out,thank you yes doing now,please help got scammed and need help finding ...,,...,mir1 flying from san francisco phoenix pocket ...,love livepage thank you much for developing re...,just signed for and got free online storage,say hello the future mail follow for hot news ...,check out personalized 2012 gaming summary fro...,mark bale feat cosmo klein gimme some love ori...,matty menck rain original mix,0.0,1980.0,4.0
1,719703,other words good news about the vaccine safety...,people forget that trump didn even want sign t...,the taiwan government response covid simply ou...,,quite happy with yesterday work zen garden jus...,yikes isn easy comprehend with all speeches fr...,harder and harder get excited about buying new...,excellent news been feeling exhausted this out...,what line,...,wow congrats very mixed feelings right now goi...,glitch was wonderful game and really miss stil...,was the game until the final few minutes could...,bringing macbook went the hotel lobby use thei...,but the flight was majorly delayed thank god f...,was going miss the last hours the game because...,tonight getting pretty emotional when think ab...,0.0,1985.0,4.0
2,749003,would fair call lil nas the first successful o...,did,decided only engage with batman and batman adj...,reading 1953 mad magazine batman parody harvey...,happy batman day batman people batman,because not constrained the suits the furnitur...,the second half robbie declares that his campa...,iffy,forgot the second wink which makes the first w...,...,this page layout makes sense want read green f...,there are 911 stans,triggered the you really want tweet that proba...,get ucked,this dystopic,this neat the motion lines show the monster fa...,love angel movie baby,0.0,1982.0,5.0
3,822540,,bonk,,nice,mcboy,oos getting real tired gamestop today,xsx sold out was removed from cart after multi...,have pro can add cart,wish they had used that album art for quite op...,...,the tradition festivus begins with the airing ...,glad spotted this finally opened sealed copy a...,you too carp,was shocked how not fun the first hours this g...,remember when newegg was known for their great...,yub nub,what happened staying feet apart from other pe...,0.0,1979.0,4.0
4,865071,how about pizza dipped water,day quarantine installed trapeze the living ro...,love portuguese egg tarts too but this really ...,well that was unexpected memory from the raver...,the has many benefits but needing constantly a...,why does nobody talk about the fact that donke...,basically put the info one needs steal identit...,the start,the start village and ready run the fort wadsw...,...,alicia busts open our shipping container carls...,who that,wow sabía tampoco que había estado coma por cu...,did leave prison review app running again,just passed sing sing the train heading north ...,omigod omigod omigod,yvr jfk,0.0,1995.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3271,3323699762,,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,1.0,1987.0,4.0
3272,3352812676,women guide burn fat and build muscle the holy...,wondering what now that the weather turning co...,the what who why and where markdown,that one the reasons these victoria secret ang...,lewis remembered his hometown belfast with the...,how are readers finding you the answer this qu...,how keep through the holidays from,should you let your dog sleep your bed you,publishing new book exciting but you need make...,...,landing interview for your book can challengin...,are apples good for health benefits feeding fr...,why kids can write,you want book here the secret,the classic dilemma build your audience focus ...,dna tests one mutt experience,what the best for guide nutritional needs heal..

In [11]:
# get the age label through year_born column
df_cleanedtweets['age'] = df_cleanedtweets['year_born'].apply(lambda x: 2021-x)
df_cleanedtweets['age'] = df_cleanedtweets['age'].apply(lambda x: 1 if x >= 21 else 0)
df_cleanedtweets

,user_id,0,1,2,3,4,5,6,7,8,...,94,95,96,97,98,99,is_female,year_born,race,age
0,12488,ykar futuristic sans serif font,who can contact about the very rude and poor s...,like win,,now heading b1000th floor,check this out,thank you yes doing now,please help got scammed and need help finding ...,,...,love livepage thank you much for developing re...,just signed for and got free online storage,say hello the future mail follow for hot news ...,check out personalized 2012 gaming summary fro...,mark bale feat cosmo klein gimme some love ori...,matty menck rain original mix,0.0,1980.0,4.0,1
1,719703,other words good news about the vaccine safety...,people forget that trump didn even want sign t...,the taiwan government response covid simply ou...,,quite happy with yesterday work zen garden jus...,yikes isn easy comprehend with all speeches fr...,harder and harder get excited about buying new...,excellent news been feeling exhausted this out...,what line,...,glitch was wonderful game and really miss stil...,was the game until the final few minutes could...,bringing macbook went the hotel lobby use thei...,but the flight was majorly delayed thank god f...,was going miss the last hours the game because...,tonight getting pretty emotional when think ab...,0.0,1985.0,4.0,1
2,749003,would fair call lil nas the first successful o...,did,decided only engage with batman and batman adj...,reading 1953 mad magazine batman parody harvey...,happy batman day batman people batman,because not constrained the suits the furnitur...,the second half robbie declares that his campa...,iffy,forgot the second wink which makes the first w...,...,there are 911 stans,triggered the you really want tweet that proba...,get ucked,this dystopic,this neat the motion lines show the monster fa...,love angel movie baby,0.0,1982.0,5.0,1
3,822540,,bonk,,nice,mcboy,oos getting real tired gamestop today,xsx sold out was removed from cart after multi...,have pro can add cart,wish they had used that album art for quite op...,...,glad spotted this finally opened sealed copy a...,you too carp,was shocked how not fun the first hours this g...,remember when newegg was known for their great...,yub nub,what happened staying feet apart from other pe...,0.0,1979.0,4.0,1
4,865071,how about pizza dipped water,day quarantine installed trapeze the living ro...,love portuguese egg tarts too but this really ...,well that was unexpected memory from the raver...,the has many benefits but needing constantly a...,why does nobody talk about the fact that donke...,basically put the info one needs steal identit...,the start,the start village and ready run the fort wadsw...,...,who that,wow sabía tampoco que había estado coma por cu...,did leave prison review app running again,just passed sing sing the train heading north ...,omigod omigod omigod,yvr jfk,0.0,1995.0,4.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3271,3323699762,,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,1.0,1987.0,4.0,1
3272,3352812676,women guide burn fat and build muscle the holy...,wondering what now that the weather turning co...,the what who why and where markdown,that one the reasons these victoria secret ang...,lewis remembered his hometown belfast with the...,how are readers finding you the answer this qu...,how keep through the holidays from,should you let your dog sleep your bed you,publishing new book exciting but you need make...,...,are apples good for health benefits feeding fr...,why kids can write,you want book here the secret,the classic dilemma build your audience focus ...,dna tests one mutt experience,what the best for guide nutritional needs heal...,1.0,1973.0,4.0,1
3273,3924536853,even though school cancelled and grades don ma...,,,,,,,,this the driest eyeliner ever used really pull...,...,this holy grail product for,cute but don use blush,pls discuss the fact that man sky under invest...,you wear makeup join,loved the skincare produ

In [12]:
df_cleanedtweets.to_csv('./data/preprocessed_tweets_with_race&age.csv')